In [1]:
from transformers import GPT2Config, TFGPT2Model
import tensorflow as tf
import numpy as np
import os

import utils
import config

ROOT_PATH = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
conf = config.Config("single_instruments_type", ROOT_PATH)

/home/marcello/github/MusicGeneration/env/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2022-11-17 18:49:10.459632: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-17 18:49:10.619520: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-17 18:49:11.180069: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open s

In [10]:
# dataset = tf.data.Dataset.load(conf.lmda_genres_tf_data_path)   \
dataset = tf.data.Dataset.load(conf.tf_data_path)   \
    .shuffle(conf.SHUFFLE_SIZE)                                 \
    .batch(conf.BATCH_SIZE)                                     \
    .prefetch(conf.PREFETCH_SIZE)                               \
    .cache()                                                    


In [11]:
class MusicGenerator(tf.keras.Model):

    def __init__(self, conf: config.Config,*args, **kwargs):
        super().__init__(*args, **kwargs)

        self.conf = conf

        self.embeddings = conf.embedding_layers
        self.pos_encoding_matrix = conf.get_positional_embedding_matrix()
        self.concat_layer = tf.keras.layers.Concatenate(axis=2)

        self.decoder = conf.get_decoder()

    
    def call(self, inputs):
        song = inputs[0]
        genre = inputs[1]

        for line in song:
            outputs = [self.embeddings[i](line[i]) for i in range(len(self.embeddings))]
            concat_outputs = self.concat_layer(outputs)
            break
        return concat_outputs




In [14]:
model = MusicGenerator(conf)

for song_batch, genre_batch in dataset.take(1):
    tf.gather(song_batch)
    notes = [tf.slice(song_batch, [0, 0, i], [-1, 0, i]) for i in range(8192)]
    print(notes[0])

2022-11-17 18:59:03.635505: W tensorflow/core/kernels/data/cache_dataset_ops.cc:856] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


InvalidArgumentError: {{function_node __wrapped__Slice_device_/job:localhost/replica:0/task:0/device:CPU:0}} Expected size[2] in [0, 5], but got 6 [Op:Slice]